In [8]:
!pip install pyts
from pyts.classification import BOSSVS
from pyts.datasets import load_basic_motions
from pyts.multivariate.classification import MultivariateClassifier


from os import system
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers import Attention
import numpy as np
import sys
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from google.colab import drive
drive.mount('/content/drive')
np.set_printoptions(threshold=sys.maxsize)
# from scipy.signal import argrelextrema
# from statsmodels.nonparametric.kernel_regression import KernelReg
def find_MaxtimeStep(df):
    x = df.loc[df['TREND'].notnull()].index.tolist()
    max_distance =0 
    for i in range (0,len(x)-1):
        if x[i+1]-x[i] >max_distance:
            max_distance = x[i+1]-x[i]
    return max_distance
def get_fixedSizeCandle(candles,md):
    candles_array = np.array(candles)
    Fixed_MaxSizeCandles = np.zeros((md,candles_array.shape[1]),dtype=float)
    Fixed_MaxSizeCandles[md-candles_array.shape[0]:,:candles_array.shape[1]] =candles_array
    return Fixed_MaxSizeCandles

def PrepareData(path):
    print ('Reading data...')
    excel_data = pd.read_excel(path)
    df = pd.DataFrame(excel_data, columns=['DATE', 'TIME', 'OPEN','HIGH','LOW','CLOSE','TREND'])
    print ('Reading data Complete!')
    print ('Creating Input Array ...')
    
    Max_TimeStep = find_MaxtimeStep(df)
    x=list()
    y=list()
    i=0
    candles = list()
    while i<len(df):
        candle =list()
        candle = np.zeros(4)
        if not(pd.isnull(df.at[i,'TREND'])):
            y.append(df.at[i,'TREND'])
            if len(candles)>0:
                    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
            candles = list()
        else:
          for j in range (0,4):
              candle[j] = df.iloc[i,j+2] -df.iloc[i-1,j+2]
            #candle[j] = df.iloc[i,j+2]
        candles.append(candle)
        i = i+1
        
    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
    x = np.array(x)
    y = np.array(y)
    scaler = StandardScaler()
    for i in range(0,x.shape[0]):
        x[i]= scaler.fit_transform(x[i])
    #x = scaler.fit_transform(x.reshape(-1, x.shape[-1])).reshape(x.shape)
    label_encoder = LabelEncoder()
    # print (y)
    # integer_encoded = label_encoder.fit_transform(y)
    # print(integer_encoded)
    # onehot_encoder = OneHotEncoder(sparse=False)
    # integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    # onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # print ('Creating Input Array Complete!')
    # return np.array(x),np.array(onehot_encoded),Max_TimeStep
    return np.array(x),np.array(y),Max_TimeStep

def Create_Model(num_timesteps,num_features):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=(num_timesteps, num_features)))
    model.add(Bidirectional(LSTM(32)))
    model.add(Dense(16))
    model.add(Dense(6,activation= 'softmax'))
    return model

def Show_history_plot(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
def Create_Train_Test(x,y,test_Persent):
 
    X_Train = x[:int((100-test_Persent)*x.shape[0]/100),:,:]
    X_Test = x[(X_Train.shape[0]):,:,:]
    Y_Train = y[:X_Train.shape[0]]
    Y_Test = y[X_Train.shape[0]:]
    return X_Train,X_Test,Y_Train,Y_Test

#set parameters
test_Persent = 15
Max_TimeStep=0
Max_TimeStep1=0
print('Preparing Data ...')
x,y,Max_TimeStep = PrepareData('/content/drive/MyDrive/Dataset/SampleTestData.xlsx')
#x1,y1, = PreparetestData('/content/drive/MyDrive/Test_Sample.xlsx',Max_TimeStep)
X_Train,X_Test,Y_Train,Y_Test = Create_Train_Test(x,y,test_Persent)
print('Data is Ready ... (sample size:')
print(X_Train.shape)
X_Train= X_Train.transpose(0,2,1)
X_Test= X_Test.transpose(0,2,1)
print(X_Train.shape)
print(Y_Train.shape)

clf = MultivariateClassifier(BOSSVS(strategy ='normal'))
clf.fit(X_Train, Y_Train)
MultivariateClassifier(...)
clf.score(X_Train, Y_Train)
a = clf.predict(X_Test)
print(X_Test.shape)
print(Y_Test)
print (a)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Preparing Data ...
Reading data...
Reading data Complete!
Creating Input Array ...
Data is Ready ... (sample size:
(8, 337, 4)
(8, 4, 337)
(8,)
(2, 4, 337)
['Break DownTrend' 'UpTrend']
['Break DownTrend' 'Side']
